In [1330]:
import requests
from bs4 import BeautifulSoup
import re
import json
import csv
import pandas as pd
import numpy as np
from pprint import pprint

In [1331]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [1332]:
filelist = ['vw_cars.csv', 'toyota_cars.csv', 'mercedes_cars.csv', 'honda_cars.csv', 'nissan_cars.csv', 'bmw_cars.csv',
           'mitsu_cars.csv', 'audi_cars.csv', 'skoda_cars.csv', 'infiniti_cars.csv', 'lexus_cars.csv', 'volvo_cars.csv']

In [1333]:
columns = ['url', 'bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'modelName', 'name', 'model_attr',
              'numberOfDoors', 'model_razgon', 'model_consumption', 'model_class', 'productionDate',
              'vehicleTransmission', 'engineDisplacement', 'enginePower', 'description', 'mileage', 'Комплектация',
              'Привод', 'Руль', 'Состояние', 'Владельцы', 'ПТС', 'Таможня', 'Владение', 'Price']

In [1334]:
data=pd.DataFrame()
for carset in filelist:
    tmp_data=pd.read_csv(carset, header=None, error_bad_lines=False)
    data = pd.concat([data, tmp_data], ignore_index=True)


b'Skipping line 575: expected 28 fields, saw 29\nSkipping line 18159: expected 28 fields, saw 65\n'


In [1335]:
data.columns=columns

In [1336]:
# Выкинем строки, которые неправильно загрузились из-за спецсимволов
#data.drop(data.index[[575, 25192, 28786, 39098]], inplace = True)

In [1337]:
data.sample(5)

,url,bodyType,brand,color,fuelType,modelDate,modelName,name,model_attr,numberOfDoors,model_razgon,model_consumption,model_class,productionDate,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,Price
51562,https://auto.ru/cars/used/sale/mercedes/s_klas...,седан,Mercedes-Benz,чёрный,Бензин,2005,Б/у Mercedes-Benz,600 Long 5.5 AT (517 л.с.),"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",4.0,4.6 с,14.3 л,F,2006.0,автоматическая,5.5 л,517 л.с.,Мерседес S600. Заводской пакет AMG. За все вре...,53000.0,NaN,задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,1 700 000 ₽
108100,https://auto.ru/cars/used/sale/mitsubishi/l200...,пикап двойная кабина,Mitsubishi,серебристый,Дизель,2006,Б/у Mitsubishi,2.5d MT (136 л.с.) 4WD,"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",4,14.6 с,8.6 л,J,2007,механическая,2.5 л,136 л.с.,"Один хозяин, честный пробег, тайская сборка, с...",317000.0,NaN,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,899 000 ₽
28050,https://auto.ru/cars/used/sale/toyota/chaser/1...,седан,Toyota,серый,Бензин,1996,Б/у Toyota,2.0 AT (140 л.с.),"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",4,NaN,9.6 л,E,1998.0,автоматическая,2.0 л,140 л.с.,"Мотор откапитален, требуется косметика. Бензин...",370000.0,NaN,задний,Правый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,283 000 ₽
58764,https://auto.ru/cars/used/sale/mercedes/m_klas...,внедорожник 5 дв.,Mercedes-Benz,коричневый,Бензин,2011,Б/у Mercedes-Benz,400 3.0 AT (333 л.с.) 4WD,"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",5.0,6.1 с,9.3 л,J,2015.0,автоматическая,3.0 л,333 л.с.,Калеса зима лето на оригинальных дисках. камер...,100000.0,"{""Комфорт"": [""Круиз-контроль"", ""Мультифункцион...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,6 лет и 1 месяц,2 500 000 ₽
108093,https://auto.ru/cars/used/sale/mitsubishi/l200...,пикап двойная кабина,Mitsubishi,чёрный,Дизель,2006,Б/у Mitsubishi,2.5d MT (136 л.с.) 4WD,"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",4,14.6 с,8.6 л,J,2008,механическая,2.5 л,136 л.с.,"С машиной идет кунг, фаркоп, усиленная защита ...",263000.0,"{""Прочее"": [""Защита картера"", ""Фаркоп""]}",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,NaN


In [1338]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139220 entries, 0 to 139219
Data columns (total 28 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   url                  139219 non-null  object
 1   bodyType             139119 non-null  object
 2   brand                139120 non-null  object
 3   color                139124 non-null  object
 4   fuelType             138661 non-null  object
 5   modelDate            139105 non-null  object
 6   modelName            139119 non-null  object
 7   name                 139118 non-null  object
 8   model_attr           139196 non-null  object
 9   numberOfDoors        139113 non-null  object
 10  model_razgon         124576 non-null  object
 11  model_consumption    116883 non-null  object
 12  model_class          137947 non-null  object
 13  productionDate       139118 non-null  object
 14  vehicleTransmission  139113 non-null  object
 15  engineDisplacement   139113 non-nu

In [1339]:
# Очистим столбец с ценой
data.Price = data.Price.str.replace(u'\xa0', u'')
data.Price = pd.to_numeric(data.Price.str.strip('₽'))

# Уберем записи без целевой переменной
data.dropna(subset=['Price'], inplace=True)
data.Price = data.Price.astype(int)

In [1340]:
# Уберем без типа кузова
data.dropna(subset=['bodyType'], inplace=True)

In [1341]:
# Обработка для электроавтомобилей
data.loc[data.name.str.contains("Electro"), 'fuelType']='электро'
data.loc[data.name.str.contains("Electro"), 'model_razgon']=data[data.name.str.contains("Electro")].model_consumption
data.loc[data.name.str.contains("Electro"), 'enginePower']=data[data.name.str.contains("Electro")].engineDisplacement
data.loc[data.name.str.contains("Electro"), 'engineDisplacement']='0 л'

In [1342]:
# Приводим виды топлива к test
data.fuelType= data.fuelType.str.replace(', газобаллонное оборудование', '')
data.fuelType= data.fuelType.str.lower()

In [1343]:
data.fuelType.unique()

array(['бензин', 'дизель', 'газ', 'гибрид', 'электро'], dtype=object)

In [1344]:
def fill_modelname(record):
    return (record.split(',')[7].split(':')[1].strip('"'))

In [1345]:
data.modelName = data.model_attr.apply(fill_modelname)

In [1346]:
def fill_segment(record):
    return (record.split(',')[12].split(':')[1].strip('"'))

In [1347]:
data['segment'] = data.model_attr.apply(fill_segment)

In [1348]:
# В числовой вид
data.modelDate = pd.to_numeric(data.modelDate)
data.productionDate = pd.to_numeric(data.productionDate)

In [1349]:
data.modelDate.unique()

array([1942., 1991., 1950., 1996., 1988., 1938., 1961., 1993., 1997.,
       1955., 1998., 1970., 1992., 1990., 1975., 1995., 1973., 1967.,
       1974., 1983., 1978., 1984., 1979., 1980., 1981., 1994.,   nan,
       2000., 1999., 2001., 2002., 2005., 2004., 2006., 2003., 2007.,
       2009., 2008., 2010., 2011., 2012., 2014., 2013., 2015., 2016.,
       2017., 2018., 2020., 2019., 1971., 1987., 1976., 1977., 1989.,
       1982., 1960., 1985., 1986., 1934., 1936., 1937., 1951., 1958.,
       1959., 1965., 1968., 1963., 1972., 1969., 1949., 1957., 1966.,
       1953., 1952.])

In [1350]:
data[data.modelDate.isna()]

,url,bodyType,brand,color,fuelType,modelDate,modelName,name,model_attr,numberOfDoors,model_razgon,model_consumption,model_class,productionDate,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,Price,segment
274,https://auto.ru/cars/used/sale/volkswagen/golf...,хэтчбек 5 дв.,Volkswagen,синий,бензин,NaN,GOLF,1.8 MT (90 л.с.),"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",NaN,NaN,NaN,NaN,1996.0,механическая,1.8 л,90 л.с.,Гольф 3 на ходу. Все вопросы по телефону,300000,NaN,передний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,NaN,105000,MEDIUM
21644,https://auto.ru/cars/used/sale/toyota/corona/1...,седан,Toyota,фиолетовый,бензин,NaN,CORONA,1.8 AT (125 л.с.),"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",4,NaN,8.1 л,D,1993.0,автоматическая,1.8 л,125 л.с.,"Звоните пишите расскажем, торг ороший",250000.0,NaN,передний,Правый,Не требует ремонта,3 или более,Оригинал,Растаможен,1 год и 3 месяца,125000,MEDIUM
28862,https://auto.ru/cars/used/sale/toyota/estima/1...,минивэн,Toyota,серебристый,бензин,NaN,ESTIMA,2.4 AT (160 л.с.) 4WD,"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",5,11 с,NaN,M,2000.0,автоматическая,2.4 л,160 л.с.,"Машина в хорошем состоянии,все расходники меня...",399000.0,NaN,полный,Правый,Не требует ремонта,1 владелец,Дубликат,Растаможен,NaN,520000,MEDIUM
35440,https://auto.ru/cars/used/sale/toyota/corolla/...,седан,Toyota,чёрный,бензин,NaN,COROLLA,1.6 AT (124 л.с.),"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",4,12.1 с,7.2 л,C,2008.0,автоматическая,1.6 л,124 л.с.,Срочна продам в стройки деньги нужни владелец ...,210000.0,"{""Комфорт"": [""Круиз-контроль""], ""Обзор"": [""Дат...",передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,439499,MEDIUM
65083,https://auto.ru/cars/used/sale/honda/jazz/1102...,хэтчбек 5 дв.,Honda,серебристый,бензин,NaN,JAZZ,1.3 MT (83 л.с.),"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",5.0,12 с,NaN,B,2002.0,механическая,1.3 л,83 л.с.,"Состояние хороший ,масло не жрёт,от заметно до...",146650.0,NaN,передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,350000,MEDIUM
65086,https://auto.ru/cars/used/sale/honda/accord/10...,седан,Honda,серый,бензин,NaN,ACCORD,2.0 AT (155 л.с.),"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",4.0,10.8 с,8.3 л,D,2003.0,автоматическая,2.0 л,155 л.с.,Продам верного друга|по кузову целый - по фото...,200000.0,NaN,передний,Правый,Не требует ремонта,3 или более,Оригинал,Растаможен,6 лет и 2 месяца,400000,MEDIUM
68809,https://auto.ru/cars/used/sale/nissan/bluebird...,седан-хардтоп,Nissan,белый,бензин,NaN,BLUEBIRD_MAXIMA,2.0 MT (115 л.с.),"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",4.0,NaN,NaN,NaN,1987.0,механическая,2.0 л,115 л.с.,Уникальный и редкий автомобиль(в городе таких ...,193000.0,NaN,передний,Правый,Не требует ремонта,3 или более,Дубликат,Растаможен,NaN,140000,MEDIUM
80560,https://auto.ru/cars/used/sale/nissan/note/110...,хэтчбек 5 дв.,Nissan,чёрный,бензин,NaN,NOTE,1.6 AT (110 л.с.),"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",NaN,NaN,NaN,NaN,2008.0,автоматическая,1.6 л,110 л.с.,Машина в нормальном состоянии. Стоит газ новог...,190469.0,"{""Салон"": [""Тонированные стекла"", ""Комбинирова...",передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,360000,MEDIUM
89210,https://auto.ru/cars/used/sale/bmw/5er/1099940...,седан,BMW,серебристый,бензин,NaN,5ER,520i 2.0 MT (150 л.с.),"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",4.0,10.2 с,9.1 л,E,2000.0,механическая,2.0 л,150 л.с.,Легендарная е39в идеальном состоянии как по ку...,216000.0,NaN,задний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,NaN,370000,PREMIUM
96194,https://auto.ru/cars/used/sale/bmw/x5/11028901...,внедорожник 5 дв.,BMW,чёрный,дизель,NaN,X5,30d 3.0d AT (245 л.с.) 4WD,"{""sale-data-attributes"":{""asciiCat"":""cars"",""ca...",5.0,7.6 с,7.4 л,J,2

In [1351]:
data.dropna(subset=['modelDate'], inplace=True)

In [1352]:
data.dropna(subset=['Состояние'], inplace=True)

In [1353]:
# В числовой вид
data.Владельцы = pd.to_numeric(data.Владельцы.str[0])

In [1354]:
data.Состояние.fillna('Не требует ремонта', inplace=True)
data.Владельцы.fillna(3, inplace=True)
data.ПТС.fillna('Оригинал', inplace=True)
data.Таможня.fillna('Растаможен', inplace=True)
data.Владельцы=data.Владельцы.astype(int)

In [1355]:
# В числовой вид
data.numberOfDoors = pd.to_numeric(data.numberOfDoors)

In [1356]:
data.numberOfDoors.unique()

array([0., 3., 5., 2., 4.])

In [1357]:
# Очистим столбец и исправим написание мерседеса
data.brand = data.brand.str.upper()
data.loc[data.brand == 'MERCEDES-BENZ', 'brand'] = 'MERCEDES'

In [1358]:
data.brand.unique()

array(['VOLKSWAGEN', 'TOYOTA', 'MERCEDES', 'HONDA', 'NISSAN', 'SUZUKI',
       'BMW', 'MITSUBISHI', 'MITSUOKA', 'AUDI', 'SKODA', 'INFINITI',
       'LEXUS', 'VOLVO'], dtype=object)

In [1359]:
data = data.loc[~data.brand.isin(['MITSUOKA', 'SUZUKI'])]

In [1360]:
data.engineDisplacement.unique()

array(['1.1 л ', '1.8 л ', '1.5 л ', '1.9 л ', '2.0 л ', '1.3 л ',
       '1.4 л ', '1.6 л ', '2.4 л ', '1.2 л ', '2.5 л ', '2.8 л ',
       '0.9 л ', '1.7 л ', '2.3 л ', '2.1 л ', '1.0 л ', '2.9 л ',
       '6.0 л ', '3.2 л ', '4.0 л ', '4.9 л ', '3.6 л ', '3.0 л ',
       '4.2 л ', '4.1 л ', '0 л', '2.6 л ', '2.2 л ', '4.5 л ', '3.4 л ',
       '2.7 л ', '3.5 л ', '4.7 л ', '3.3 л ', '4.3 л ', '5.7 л ',
       '4.6 л ', '5.0 л ', '0.7 л ', '6.8 л ', '3.8 л ', '5.8 л ',
       '5.4 л ', '5.6 л ', '5.5 л ', '6.2 л ', '3.7 л ', '4.4 л ',
       '4.8 л ', '0.6 л ', '3.9 л ', '6.6 л ', '3.1 л ', '5.2 л ',
       '5.9 л ', '6.3 л '], dtype=object)

In [1361]:
# Очистим столбец с объемом двигателя
data.engineDisplacement = pd.to_numeric(data.engineDisplacement.str.strip(' л'))

In [1362]:
data.enginePower.unique()

array(['25\xa0л.с.', '75\xa0л.с.', '42\xa0л.с.', '110\xa0л.с.',
       '90\xa0л.с.', '115\xa0л.с.', '40\xa0л.с.', '150\xa0л.с.',
       '45\xa0л.с.', '44\xa0л.с.', '101\xa0л.с.', '80\xa0л.с.',
       '48\xa0л.с.', '68\xa0л.с.', '34\xa0л.с.', '78\xa0л.с.',
       '50\xa0л.с.', '193\xa0л.с.', '140\xa0л.с.', '55\xa0л.с.',
       '60\xa0л.с.', '47\xa0л.с.', '72\xa0л.с.', '116\xa0л.с.',
       '54\xa0л.с.', '70\xa0л.с.', '85\xa0л.с.', '174\xa0л.с.',
       '102\xa0л.с.', '79\xa0л.с.', '95\xa0л.с.', '84\xa0л.с.',
       '125\xa0л.с.', '57\xa0л.с.', '107\xa0л.с.', '139\xa0л.с.',
       '112\xa0л.с.', '58\xa0л.с.', '136\xa0л.с.', '88\xa0л.с.',
       '100\xa0л.с.', '105\xa0л.с.', '69\xa0л.с.', '83\xa0л.с.',
       '160\xa0л.с.', '64\xa0л.с.', '120\xa0л.с.', '129\xa0л.с.',
       '170\xa0л.с.', '87\xa0л.с.', '130\xa0л.с.', '204\xa0л.с.',
       '65\xa0л.с.', '98\xa0л.с.', '180\xa0л.с.', '190\xa0л.с.',
       '420\xa0л.с.', '241\xa0л.с.', '275\xa0л.с.', '220\xa0л.с.',
       '313\xa0л.с.', '46\x

In [1363]:
# Очистим столбец с мощностью двигателя
data.enginePower=data.enginePower.str.strip()
data.enginePower=data.enginePower.str.strip('\xa0л.с.')
data.enginePower=pd.to_numeric(data.enginePower).astype(int)

In [1364]:
# В числовой вид
data.mileage = pd.to_numeric(data.mileage)

In [1365]:
# Заменим опции в data на аналогичные названия в test
options_subst = pd.read_excel('options_subst1.xls')
data.Комплектация = data.Комплектация.str.replace('(','')
data.Комплектация = data.Комплектация.str.replace(')','')

data_option = list(options_subst['data option'])
new_option = list(options_subst['subst'])

for i in range(len(data_option)):
    data.Комплектация = data.Комплектация.str.replace(data_option[i], new_option[i])

C:\Users\maxsv\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\maxsv\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
C:\Users\maxsv\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


In [1366]:
# Конвертируем "строковые" словари с опциями автомобилей в data в простые списки опций
def data_option_fix(record):
    options = []
    if type(record)==str:
        options_dict = json.loads(record)
        for key in options_dict:
            options.extend(options_dict[key])
    return(options)

In [1367]:
data.Комплектация = data.Комплектация.apply(data_option_fix)

In [1368]:
#Получаем список уникальных опций в data
#b = []
#for i in range(1, len(data.Комплектация.unique())):
#    a = json.loads(data.Комплектация.unique()[i])
#    for key in a:
#        for el in a[key]:
#            if el not in b:
#                b.append(el)

In [1369]:
# sorted(b)

In [1370]:
# Создаем столбец VENDOR аналогичный test
def vendor_create(record):
    eur = ['SKODA', 'AUDI', 'VOLVO', 'BMW', 'MERCEDES', 'VOLKSWAGEN']
    jap = ['HONDA', 'NISSAN', 'INFINITI', 'TOYOTA', 'LEXUS', 'MITSUBISHI']
    if record in eur:
        return ('EUROPEAN')
    elif record in jap:
        return ('JAPANESE')
    else:
        return(None)
    
data['vendor'] = data.brand.apply(vendor_create)

In [1371]:
data.model_razgon=pd.to_numeric(data.model_razgon.str.strip('с'))

In [1372]:
data.model_razgon.isna().sum()

14623

In [1373]:
# Заполним отсутствующий разгон средним значением для марки, модели автомобиля и года выпуска
avg_razgon = pd.DataFrame(data.groupby(by=['brand', 'modelName', 'enginePower']).mean()['model_razgon']).reset_index()
avg_razgon.columns = ['brand', 'modelName', 'enginePower','avg_razgon']

data = data.merge(avg_razgon, on=['brand', 'modelName', 'enginePower'], how='left')
data.loc[data.model_razgon.isna(), 'model_razgon'] = data[data.model_razgon.isna()].avg_razgon

In [1374]:
# Убираем промежуточные колонки
data.drop(['avg_razgon'], axis=1, inplace=True)

In [1375]:
# Заполним отсутствующий разгон средним значением для мощности двигателя
avg_razgon = pd.DataFrame(data.groupby(by=['enginePower']).mean()['model_razgon']).reset_index()
avg_razgon.columns = ['enginePower','avg_razgon']

data = data.merge(avg_razgon, on=['enginePower'], how='left')
data.loc[data.model_razgon.isna(), 'model_razgon'] = data[data.model_razgon.isna()].avg_razgon

In [1376]:
# Убираем промежуточные колонки
data.drop(['avg_razgon'], axis=1, inplace=True)

In [1377]:
# Заполним отсутствующий разгон средним значением для модели
avg_razgon = pd.DataFrame(data.groupby(by=['modelName']).mean()['model_razgon']).reset_index()
avg_razgon.columns = ['modelName','avg_razgon']

data = data.merge(avg_razgon, on=['modelName'], how='left')
data.loc[data.model_razgon.isna(), 'model_razgon'] = data[data.model_razgon.isna()].avg_razgon

In [1378]:
# Убираем промежуточные колонки
data.drop(['avg_razgon'], axis=1, inplace=True)

In [1379]:
sorted(data[data.model_razgon.isna()].enginePower.unique())

[7, 19, 25, 38, 42, 44, 46, 47, 51, 74, 336]

In [1380]:
data.loc[data.model_razgon.isna() & (data.enginePower == 336), 'model_razgon'] = 6
data.loc[data.model_razgon.isna() & (data.enginePower < 80), 'model_razgon'] = 20

In [1381]:
data.model_consumption=pd.to_numeric(data.model_consumption.str.strip('л'))

In [1382]:
data.model_consumption.unique()

array([  nan,  7.4 ,  6.7 ,  7.5 ,  8.2 ,  8.1 ,  8.  ,  7.9 ,  6.8 ,
        7.7 , 13.5 ,  6.  ,  6.3 ,  9.5 ,  8.8 , 15.8 ,  6.5 ,  8.3 ,
        7.  ,  7.8 ,  5.5 ,  6.9 ,  9.6 , 10.  ,  9.3 ,  6.6 ,  8.4 ,
        9.  ,  5.2 ,  7.3 , 11.4 ,  5.8 ,  5.  , 11.  ,  8.6 ,  9.2 ,
        8.7 , 10.2 ,  6.2 ,  8.5 ,  8.9 , 10.1 ,  4.4 ,  4.7 ,  6.1 ,
        9.4 ,  4.8 ,  5.1 ,  4.9 ,  7.6 ,  9.8 , 12.2 ,  9.9 ,  9.1 ,
       11.8 ,  7.2 , 12.7 ,  5.7 , 12.6 , 10.9 ,  5.6 , 10.5 ,  5.9 ,
       13.  ,  5.4 , 11.6 ,  6.4 ,  5.3 , 11.1 , 11.7 ,  7.1 ,  9.7 ,
       14.7 , 15.6 , 12.1 , 13.2 , 12.8 , 11.9 , 10.3 , 12.4 , 12.9 ,
       13.8 , 14.9 , 13.3 , 14.8 , 10.8 , 10.6 , 12.5 ,  4.5 , 15.7 ,
       10.4 , 15.9 ,  4.3 ,  4.1 ,  4.  ,  4.6 ,  4.2 ,  1.5 , 17.  ,
       14.6 , 14.  , 10.7 , 16.3 , 23.8 , 12.3 , 15.5 , 11.3 , 12.  ,
       17.8 , 11.2 , 17.5 , 13.7 , 16.6 , 14.4 , 13.6 , 13.1 , 14.1 ,
       16.8 ,  3.9 , 13.4 , 16.7 ,  3.8 ,  2.7 ,  2.1 , 15.3 , 15.  ,
       13.9 ,  3.  ,

In [1383]:
data.model_consumption.isna().sum()

21542

In [1384]:
avg_consumption = pd.DataFrame(data.groupby(
    by=['brand', 'modelName', 'engineDisplacement']).mean()['model_consumption']).reset_index()
avg_consumption

,brand,modelName,engineDisplacement,model_consumption
0,AUDI,100,1.6,9.600000
1,AUDI,100,1.8,7.235802
2,AUDI,100,1.9,NaN
3,AUDI,100,2.0,7.928462
4,AUDI,100,2.1,10.850000
...,...,...,...,...
1907,VOLVO,XC90,2.4,8.821348
1908,VOLVO,XC90,2.5,11.393706
1909,VOLVO,XC90,2.9,12.900000
1910,VOLVO,XC90,3.2,11.914286


In [1385]:
# Заполним отсутствующий расход средним значением для марки, модели автомобиля и года выпуска
avg_consumption = pd.DataFrame(data.groupby(by=['brand', 'modelName', 'enginePower']).mean()['model_consumption']).reset_index()
avg_consumption.columns = ['brand', 'modelName', 'enginePower','avg_consumption']

data = data.merge(avg_consumption, on=['brand', 'modelName', 'enginePower'], how='left')
data.loc[data.model_consumption.isna(), 'model_consumption'] = data[data.model_consumption.isna()].avg_consumption
data.drop(['avg_consumption'], axis=1, inplace=True)

In [1386]:
# # Заполним отсутствующий расход средним значением для мощности двигателя
avg_consumption = pd.DataFrame(data.groupby(by=['enginePower']).mean()['model_consumption']).reset_index()
avg_consumption.columns = ['enginePower','avg_consumption']

data = data.merge(avg_consumption, on=['enginePower'], how='left')
data.loc[data.model_consumption.isna(), 'model_consumption'] = data[data.model_consumption.isna()].avg_consumption
data.drop(['avg_consumption'], axis=1, inplace=True)

In [1387]:
# # Заполним отсутствующий расход средним значением для модели
avg_consumption = pd.DataFrame(data.groupby(by=['modelName']).mean()['model_consumption']).reset_index()
avg_consumption.columns = ['modelName','avg_consumption']

data = data.merge(avg_consumption, on=['modelName'], how='left')
data.loc[data.model_consumption.isna(), 'model_consumption'] = data[data.model_consumption.isna()].avg_consumption
data.drop(['avg_consumption'], axis=1, inplace=True)

In [1388]:
sorted(data[data.model_consumption.isna()].enginePower.unique())

[7, 19, 25, 38, 42, 44, 47, 51, 321]

In [1389]:
data.model_consumption.fillna(20, inplace=True)

In [1390]:
data[data.model_consumption.isna()]

,url,bodyType,brand,color,fuelType,modelDate,modelName,name,model_attr,numberOfDoors,model_razgon,model_consumption,model_class,productionDate,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,Price,segment,vendor


In [1391]:
data[data.model_class.isna()].modelDate.unique()

array([1942., 1950., 1961., 1955., 1970., 1992., 1990., 1991., 1967.,
       1979., 1995., 2004., 2003., 2009., 2010., 2015., 1971., 1974.,
       1989., 1982., 1986., 1994., 1996., 2000., 2019., 1951., 2012.,
       2014., 1999., 1969., 1993., 1997., 1937., 1936., 1949., 1957.,
       1965., 1938., 2017., 1952., 1959., 1976.])

In [1392]:
data.model_class.isna().sum()

1137

In [1393]:
df_class = pd.DataFrame(data.groupby(by=['brand', 'modelName'])['model_class'].agg(lambda x: x.mode())).reset_index()
df_class.columns = ['brand', 'modelName', 'mode_class']
df_class

,brand,modelName,mode_class
0,AUDI,100,E
1,AUDI,200,E
2,AUDI,80,D
3,AUDI,90,D
4,AUDI,920,[]
...,...,...,...
653,VOLVO,V90_CROSS_COUNTRY,E
654,VOLVO,XC40,J
655,VOLVO,XC60,J
656,VOLVO,XC70,D


In [1394]:
# Заполним отсутствующий класс значением для марки, модели автомобиля
df_class = pd.DataFrame(data.groupby(by=['brand', 'modelName'])['model_class'].agg(lambda x: x.mode())).reset_index()
df_class.columns = ['brand', 'modelName', 'mode_class']

data = data.merge(df_class, on=['brand', 'modelName'], how='left')
data.loc[data.model_class.isna(), 'model_class'] = data[data.model_class.isna()].mode_class
data.drop(['mode_class'], axis=1, inplace=True)


In [1397]:
data.model_class.unique()

array([nan, 'C', 'D', 'M', 'B', 'J', 'A', 'S', 'F', 'E'], dtype=object)

In [1396]:
data.model_class=data.model_class.str.strip()

In [1317]:
data.model_class.fillna('C', inplace=True)

In [1318]:
data[data.model_class.isna()]

,url,bodyType,brand,color,fuelType,modelDate,modelName,name,model_attr,numberOfDoors,model_razgon,model_consumption,model_class,productionDate,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,Price,segment,vendor


In [1319]:
data.drop(['model_attr'], axis=1, inplace=True)

In [1320]:
data.columns

Index(['url', 'bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'modelName', 'name', 'numberOfDoors', 'model_razgon', 'model_consumption', 'model_class', 'productionDate', 'vehicleTransmission', 'engineDisplacement', 'enginePower', 'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние', 'Владельцы', 'ПТС', 'Таможня', 'Владение', 'Price', 'segment', 'vendor'], dtype='object')

In [1321]:
columns_new = ['car_url', 'bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'model_name', 'name', 'numberOfDoors', 'model_razgon', 'model_consumption', 'model_class', 'productionDate', 'vehicleTransmission',
               'engineDisplacement', 'enginePower', 'description', 'mileage', 'equipment_dict', 'Привод', 'Руль', 'Состояние', 'Владельцы', 'ПТС', 'Таможня', 'Владение', 'Price', 'segment', 'vendor']

In [1322]:
data.columns=columns_new

In [1323]:
columns_new = [
    'bodyType',
    'brand',
    'car_url',
    'color',
    'description',
    'engineDisplacement',
    'enginePower',
    'equipment_dict',   
    'fuelType',
    'mileage',
    'modelDate', 
    'model_name',
    'name',
    'numberOfDoors',
    'productionDate',
    'vehicleTransmission',  
    'vendor',
    'Владельцы', 
    'Владение',
    'ПТС',
    'Привод',
    'Руль',
    'Состояние',
    'Таможня', 
    'Price', 
    'model_razgon',
    'model_consumption',
    'model_class', 
    'segment',
    ]

In [1324]:
 data = data[columns_new] 

In [1325]:
data.segment.unique()

array(['ECONOMY', 'MEDIUM', 'PREMIUM', 'cars'], dtype=object)

In [1326]:
data.loc[data.segment=='cars', 'segment']='ECONOMY'

In [1327]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132910 entries, 0 to 132909
Data columns (total 29 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   bodyType             132910 non-null  object 
 1   brand                132910 non-null  object 
 2   car_url              132910 non-null  object 
 3   color                132910 non-null  object 
 4   description          127866 non-null  object 
 5   engineDisplacement   132910 non-null  float64
 6   enginePower          132910 non-null  int32  
 7   equipment_dict       132910 non-null  object 
 8   fuelType             132910 non-null  object 
 9   mileage              132910 non-null  float64
 10  modelDate            132910 non-null  float64
 11  model_name           132910 non-null  object 
 12  name                 132910 non-null  object 
 13  numberOfDoors        132910 non-null  float64
 14  productionDate       132910 non-null  float64
 15  vehicleTransmissi

In [1328]:
pd.DataFrame(data.model_name.value_counts()).T

,PASSAT,5ER,E_KLASSE,CAMRY,OCTAVIA,POLO,COROLLA,3ER,LANCER,A6,TIGUAN,C_KLASSE,LAND_CRUISER,X5,RAV_4,QASHQAI,X_TRAIL,A4,GOLF,LAND_CRUISER_PRADO,TOUAREG,OUTLANDER,ALMERA,S_KLASSE,JETTA,RAPID,M_KLASSE,PRIMERA,PAJERO,ACCORD,RX,7ER,X6,CIVIC,TEANA,100,Q7,80,X3,CR_V,MARK_II,AVENSIS,JUKE,GL_KLASSE,GALANT,TRANSPORTER,FABIA,ALMERA_CLASSIC,CORONA,TERRANO,MURANO,SUNNY,LX,VITO,YETI,CARINA,PAJERO_SPORT,X1,XC90,SUPERB,CARISMA,NOTE,Q5,1ER,A3,W124,TIIDA,ASX,A8,PRIUS,CARAVELLE,PATROL,FX,A5,HIGHLANDER,HR_V,A_KLASSE,GLK_KLASSE,G_KLASSE,VITZ,MULTIVAN,CALDINA,MAXIMA,LEAF,PASSAT_CC,S80,S60,PATHFINDER,Q3,CROWN,FIT,SPRINTER,GLC_KLASSE,VISTA,WINGROAD,L200,SERENA,BLUEBIRD,CRESTA,HIACE,CEFIRO,CADDY,NX,GS,XC60,GLA_CLASS,AURIS,KODIAQ,ES,HILUX,TOWN_ACE,MARCH,CHASER,CARINA_E,GLS_KLASSE,CLA_KLASSE,CLS_KLASSE,AD,COLT,S40,6ER,GLE_KLASSE,TOURAN,AMAROK,XC70,B_KLASSE,GOLF_PLUS,LS,WISH,HARRIER,CIVIC_FERIO,G_KLASSE_AMG,GLE_KLASSE_COUPE,X4,A7,IS,W201,SHARAN,ESTIMA,ODYSSEY,SKYLINE,VIANO,GX,LITE_ACE,BORA,STEPWAGON,YARIS,PULSAR,ALLION,ALLROAD,AVENIR,V_KLASSE,CELICA,G35,BLUEBIRD_SYLPHY,STREAM,DELICA,MICRA,PROBOX,W123,IPSUM,LIBERTY,QX70,CUBE,CLK_KLASSE,HILUX_SURF,AQUA,RVR,QX56,X6_M,NAVARA,LAUREL,NOAH,PASSO,LIBERO,QX80,FREED,PLATZ,S_KLASSE_AMG,M,VOXY,850,4,CL_KLASSE,FELICIA,GLC_COUPE,EX,DIAMANTE,PRESAGE,M_KLASSE_AMG,COROLLA_SPACIO,Q50,JAZZ,TUNDRA,ROOMSTER,LARGO,FORTUNER,PREMIO,VENTO,TORNEO,MONTERO_SPORT,FUNCARGO,CORSA,SENTRA,MIRAGE,940,QX50,X7,CORONA_EXIV,IST,QASHQAI_PLUS_2,X5_M,GRANDIS,I3,RNESSA,EXPERT,ORTHIA,VANETTE,PRIUS_ALPHA,RAUM,CHARIOT,INTEGRA,PARTNER,INSPIRE,SPRINTER_MARINO,E_KLASSE_AMG,SPACE_STAR,SUCCEED,SPRINTER_CARIB,TINO,TT,VENZA,ECLIPSE,MONTERO,CEDRIC,PAJERO_IO,SHUTTLE,STARLET,ALPHARD,QX60,SPACE_WAGON,CARINA_ED,AIRTREK,PILOT,TERCEL,R_KLASSE,4RUNNER,PRESEA,VERSO,WINDOM,...,CLA_KLASSE_AMG,VIGOR,RASHEEN,ARMADA,PRIMASTAR,CONCERTO,STAGEA,CT,FIT_ARIA,PAJERO_JUNIOR,GT_R,2ACTIVETOURER,SOARER,AMG_GLC_COUPE,I,440,VANEO,GRAND_HIACE,URVAN,AVENSIS_VERSO,IQ,M6,MINICAB,CIMA,CAMRY_SOLARA,GOLF_R,760,MR2,SC,V60_CROSS_COUNTRY,TYPE_1,LIFE,FR_V,R8,GRACE,MASTER_ACE_SURF,SUPRA,JUKE_NISMO,LUCINO,UX,V50,VOLTZ,TERRANO_REGULUS,MISTRAL,GT86,BREVIS,CROSSROAD,FAIRLADY_Z,SL_KLASSE_AMG,SAI,CIVIC_TYPE_R,LATIO,TACOMA,E_TRON,AYGO,S4,OTTI,GRANVIA,PRONARD,M2,PASSO_SETTE,M4,S60_CROSS_COUNTRY,TOPPO,I8,A2,KIX,Z3,340,RC,321,C70,A_KLASSE_AMG,ELEMENT,QX30,FOX,RS7,300_SERIES,COMFORT,Q,PIXIS_EPOCH,JADE,MARK_X_ZIO,EDIX,MR_S,GTO,RS4,W136,TOURING_HIACE,CR_Z,ENDEAVOR,VANGUARD,DAYZ_ROOX,CLIPPER,REGIUSACE,SCEPTER_SEDAN,EQC,W115,LEOPARD,S2000,FTO,Q30,HOMY,350Z,V90,LANCER_RALLIART,EMERAUDE,SANTANA,LANCER_CARGO,DELICA_D2,I_MIEV,ASPIRE,SLS_AMG,DUALIS,INTEGRA_SJ,FAVORIT,HS,CORRADO,S3,3000_GT,V60,XTERRA,190_SL,ROGUE,CELSIOR,ROOMY,S7,Q3_SPORTBACK,WILL_CYPHA,SQ5,BLADE,MAYBACH_GLS,TITAN,FABIA_RS,W111,PIXIS_SPACE,TANK,PRIUS_C,PASEO,ODYSSEY_NA,EK_SPORT,W128,DEBONAIR,TTS,COUPE,EOS,CENTURY,CALIFORNIA,PASSAT_NA,SIGMA,V8,Q60,MDX,EK_CUSTOM,300ZX,YARIS_VERSO,DELICA_D3,N_ONE,ZEST,POLO_GTI,TYPE_2,VAMOS,PRESIDENT,PIXIS_VAN,326,Z,SLK_KLASSE_AMG,W186,W108,JEEP_J,ROOX,CLIPPER_RIO,QUEST,X3_M,200SX,SPARKY,PINO,CHERRY,CR_X,PROACE,METRIS,120,THATS,EK_CLASSIC,FCX_CLARITY,MEGA_CRUISER,LC,W110,GOLF_COUNTRY,100NX,W114,1200,Z3M,CRESSIDA,RS3,ARTEON,STRADA,CITAN,SLC_KLASSE_AMG,NV350_CARAVAN,SQ7,W142,K70,SLR_KLASSE,PASSPORT,SQ8,Z8,W188,W121,SERA,LIBERTA_VILLA,GOLF_SPORTSVAN,2000_C_CS,ILTIS,BLIZZARD,AVANZA,BEAT,Z1,TYPE_3,ROUTAN,W29,KARMANN_GHIA,GS_F,VERSO_S,EK_SPACE,LANGLEY,600,IS_F,VERSA,PIXO,280ZX,370Z,SLC_KLASSE,STANZA,920,COMS,Q2,GALANT_FORTIS,CLK_KLASSE_AMG,TODAY,GOLF_R32,UP,N_BOX_SLASH,TYPE_166,MIRAI,RAIZE,EXA,02,LM,220_W187
model_name,4285,3705,3553,3486,3450,3337,3302,2692,2508,2383,2265,2240,2132,2100,2047,2025,1918,1757,1735,1701,1583,1445,1435,1426,1393,1270,1207,1200,1163,1130,1111,1081,1069,1020,932,909,885,867,862,853,789,785,779,760,760,756,729,679,664,643,632,626,618,602,600,589,563,554,534,524,521,517,517,510,505,498,490,479,477,468,462,462,457,431,429,411,406,401,394,389,385,375,372,370,369,365,359,359,348,345,344,344,337,335,3

In [1329]:
# Сохраняем датасет

data.to_csv('cars_data.csv', index=False)

In [ ]:
#####################################################################################################
###################### I AM HERE #####################################################################




In [ ]:
# Очищаем столбец Владение, выделяем годы и месяцы и переводим в месяцы
def ownership_fix(record):
    if record == None:
        return(None)
    digits = re.findall(r'\d+', record)
    letters = re.findall(r'\D+', record)
    if len(digits) == 2:
        return (int(digits[0])*12 + int(digits[1]))
    elif len(digits) == 1:
        if 'го' in letters[0] or 'ле' in letters[0]:
            return(int(digits[0])*12)
        if 'ме' in letters[0]:
            return(int(digits[0]))

In [ ]:
data.Владение.fillna('', inplace=True)
data.Владение = data.Владение.apply(ownership_fix)
#data.Владение = data.Владение.astype(int)